In [1]:
import os
%pwd

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\assignment\\ETE_ML_Project\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\assignment\\ETE_ML_Project'

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str


In [19]:
from Mlproject.utils.common import *
from Mlproject.constants import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )
        return model_evaluation_config

In [25]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    log_loss,
    confusion_matrix
)

import pandas as pd

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metric(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred)
        log_loss_value = log_loss(actual, pred)
        conf_matrix = confusion_matrix(actual, pred)

        return accuracy, precision, recall, f1, roc_auc, log_loss_value,conf_matrix

    def evaluate(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        y_pred = model.predict(X_test)
        accuracy, precision, recall, f1, roc_auc, log_loss_value,conf_matrix = self.eval_metric(y_test,y_pred)

        scores = {'accuracy score':accuracy, 'Precision score':precision, 'Recall score':recall,
                  'F1 score':f1, 'ROC score':roc_auc, 'Log loss':log_loss_value}
        
        save_json(path = Path(self.config.metric_file_name), data = scores)




In [26]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-08-21 18:46:31,751: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-21 18:46:31,756: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-21 18:46:31,764: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-21 18:46:31,766: INFO: common: created directory at: artifacts]
[2024-08-21 18:46:31,768: INFO: common: created directory at: artifacts/model_evaluation]
[2024-08-21 18:46:31,850: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
